## Neural Network Training

This file is for training a standard neural network architecture for binary classification

##### 0. Imports

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split

##### 1. Load dataset

In [ ]:
data = torch.load('data.pt')
X = data['features']
y = data['labels']

FileNotFoundError: [Errno 2] No such file or directory: 'data.pkl'